# Zpracování a analýza trajektorie molekulární dynamiky
**(pseoudorealistický scénář použití `numpy` na skutečných datech)**

[Molekulární dynamika](https://en.wikipedia.org/wiki/Molecular_dynamics) je dlouho a široce rozšířená simulační technika, která dává náhled na chování 
biomolekul, materiálů, atd. 
Atomy aproximuje jako hmotné body, pracuje s jejich vzájemnou silovou interakcí, a jejich pohyb počítá podle běžných fyzikálních zákonů (Newtonovy pohybové rovnice).

Primárním výsledkem simulace je _trajektorie_, série velkého počtu snímků, každý snímek obsahuje 3D souřadnice všech atomů v systému.
Pro numerické zpracování je to tenzor $N\times M\times 3$, kde $N$ je počet snímků a $M$ počet atomů.

In [ ]:
import numpy as np
import mdtraj as md
import nglview as nv
import matplotlib.pyplot as plt

In [ ]:
data = '../data/'
pdb = data + 'trpcage.pdb'
xtc = data + 'trpcage_messed.xtc'

In [ ]:
traj = md.load_xtc(xtc, top=pdb)

In [ ]:
nv.show_mdtraj(traj)

In [ ]:
traj.xyz.shape

## Vycentrování snímků
Molekula v trajektorii nespojitě "skáče" (simulace typicky ukládá snímek jen každých 5-50 tis. vlastních kroků), vzájemně je tedy těžko srovnáme.

V prvním kole vytvořte novou trajektorii a nastavte jí pozice atomů tak, aby těžiště molekuly zůstávalo v počtátku souřadné soustavy.

In [ ]:
traj_center = md.load_pdb(pdb)

In [ ]:

# TODO: traj_center.xyz = vycentrované souřadnice  (np.array o rozměrech N x M x 3)
# ODOT

In [ ]:
v = nv.show_mdtraj(traj_center)
v

In [ ]:
# nastavení konkrétního snímku ve vizualizaci v předchozí buňce
v.frame = 1

## Rotace minimalizující RMSD

Molekulu máme ukotvenou v počátku, ale pořád se ještě volně otáčí. Pro každý snímek proto hledáme rotaci $R$, která bude minimalizovat průměrnou vzdálenost všech atomů (RMSD) vůči počátečnímu tvaru:

$$\textit{RMSD}(\textbf x, \textbf Ry) = \sqrt{ \frac1M \sum_{i=0}^{M-1} \|Ry_i - x_i \|^2}$$

Jednou z možností nalezení takové rotace je [Kabschův algoritmus](https://en.wikipedia.org/wiki/Kabsch_algorithm):

1. Vycházíme z vycentrovaných souřadnic, $\textbf x, \textbf y$ chápeme jako matice $M\times 3$. Spočteme součin $$ H = \textbf x^T \textbf y $$
a jeho rozklad na singulární hodnoty $$ U\Sigma V^T = H $$
2. Ortogonální matice $U,V$ mohou být rotace (pak $\det U = 1$) nebo kombinace rotace a reflexe ($\det U = -1$), spočteme součin $$ d = \det U  \det V$$
3. Rotace $$ R = U \left(\begin{array}{ccc}1&0&0\\ 0&1&0 \\ 0&0&d\end{array}\right) V^T$$ minimalizuje $\textit{RMSD}(\textbf x, \textbf Ry)$ (viz reference na uvedené stránce)
 

### Jeden snímek
Pro větší názornost implementujte algoritmus pro první dva snímky trajektorie, tj. hledáme vhodnou rotaci `R1`, která `xyz[1]` otočí na `xyz[0]`

In [ ]:
xyz = traj_center.xyz
xyz.shape

In [ ]:
# TODO: R1 = rotace snímku [1], matice 3 x 3


In [ ]:
# ODOT

In [ ]:
# Rotaci aplikujeme (vektory souřadnic je třeba chápat jako sloupcové, proto transopozice)
xyz1a = (R1 @ xyz[1].T).T

In [ ]:
# RMSD původní a po rotaci
def rmsd(x,y):
    return np.sqrt(np.mean(np.linalg.norm(x-y,axis=1)))

rmsd(xyz[0],xyz[1]), rmsd(xyz[0],xyz1a)

In [ ]:
# Umělá trajektorie z dvou snímků pro vizuální ověření 
traj1a = md.load_pdb(pdb)
traj1a.xyz = np.stack([xyz[0],xyz1a])

In [ ]:
nv.show_mdtraj(traj1a)

### Všechny snímky současně

Pamatujme přikázání "Nenapíšeš cyklus", místo toho využijeme vektorové/maticové operace, broadcast a `np.moveaxis` v roli zobecněné transpozice.

Ve skutečnosti je toto rozšíření (při správně napsaném algoritmu pro jeden snímek) velmi přímočaré.

In [ ]:
# TODO: R = rotace pro všechny snímky, N x 3 x 3


In [ ]:

# ODOT

In [ ]:
xyzt = np.moveaxis(xyz,1,2)
xyzt.shape

In [ ]:
xyza = R @ xyzt
xyza.shape

In [ ]:
traja = md.load_pdb(pdb)
traja.xyz = np.moveaxis(xyza,2,1)

In [ ]:
nv.show_mdtraj(traja)

## Vzdálenost atomů od výchozí pozice
S využitím broadcastu (a bez cyklů) vypočítejte současně vzdálenosti všech atomů od jejich výchozí pozice.

Vyzkoušejte pro původní i transofrmovanou trajektorii, demonstrujeme tak smysluplnost minimalizace RMSD.

In [ ]:
# TODO: dist0 = vzdálenosti všech atomů od snímku [0], N x M


In [ ]:
#ODOT

In [ ]:
# Zobrazíme vývoj vzdáleností několika atomů na prvních 200 snímcích
plt.figure(figsize=(15,4))
for i in range(0,dist0.shape[1],28):
    plt.plot(dist0[:200,i],label=str(i))
plt.legend()
plt.show()

## Vzdálenosti C$\alpha$ každý s každým

Uhlíik C$\alpha$ je "centrální" atom každé aminokyseliny. Jejich rozmístění už s velkou přesností určuje celou strukturu proteinu, proto je matice jejich vzájemných vzdáleností charakteristická pro konkrétní tvar proteinu.

Spočítejte pro každý snímek všechny jejich vzdálenosti současně, vhodným využitím broadcastu s nápovědou přes `np.newaxis`.

Pro názornost můžete postup nejprve implementovat pro první snímek a pak rozšířit.

In [ ]:
# indexy všech CA
cas = traja.topology.select('name == CA')
cas

In [ ]:
# trajektorie pouze pro CA
xyz_ca = traja.xyz[:,cas,:]
xyz_ca.shape

In [ ]:
#TODO distca = ... pole N x 20 x 20 vzdáleností


In [ ]:
#ODOT

In [ ]:
# Grafy pro některé dvojice a prvních 500 snímků
plt.figure(figsize=(15,4))
for i in range(0,cas.shape[0] - 1,4):
    plt.plot(distca[::500,i,i+1],label=f'{i} - {i+1}')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(15,4))
for i in range(0,cas.shape[0] - 1,4):
    plt.plot(distca[::500,0,i+1],label=f'{0} - {i+1}')
plt.legend()
plt.show()

## Dihedrální úhly

Bonusový úkol, spočítejme dihedrální úhly podél páteře proteinu.

In [ ]:
traj.topology.select('backbone and name != O')

## Konvergence trajektorie

Zpravidla nás zajímají trajektorie MD, které _zkonvergovaly_, tj. už vyčerpaly všechno možné chování proteinu a další pohyby už jsou jen jejich opakováním.

Jedna z možností, jak to určit je přepočítat pohyby molekuly na _esenciální souřadnice_ -- takové lineární kombinace souřadnic atomů, že dohromady "vysvětlují"
část dynamiky. Pokud význam těchto kombinací je stejný, spočteme-li ho pro celou trajektorii i její úvodní část, lze čekat, že pokračováním simulace už se nic dalšího zajímavého nestane.

Esenciální souřadnice a jejich význam zjistíme rozkladem korelační matice souřadnic přes všechny snímky na vlastní hodnoty:

1. Všechny 3D souřadnice zploštíme do matice $\textbf x$ o rozměrech $N \times 3M$
3. Spočteme korelační matici pohybů všech souřadnic $C = (\textbf x - \mu)^T \textbf (x-\mu) / N$, kde $\mu$ jsou střední hodnoty všech souřadnic
4. Spočteme absolutní vlastní hodnoty $|\lambda_i|$ matice $C$, normalizujeme je tak, aby $\sum|\lambda_i| = 1$ a setřídíme je od největší
6. Pro větší názornost vykreslíme graf kumulativního součtu vlastních hodnot do cca. 95% (dál už jsou nezajímavé)
7. Postup opakujeme pouze pro část trajektorie a grafy srovnáme
   
Postup a argumentace pochází z klasického článku https://doi.org/10.1002/prot.340170408

In [ ]:
# zploštění matice souřadnic
xyz_flat = np.reshape(traja.xyz,(traja.xyz.shape[0],-1))
xyz_flat.shape

In [ ]:
# TODO: full_cor = matice korelace pohybů souřadnic, 3M x 3M 


In [ ]:
# TODO: full_ev = vektor 3M vlastních hodnot
# ODOT
full_ev /= np.sum(full_ev) # normalizace
full_ev = np.flip(full_ev) # opačné pořadí
cumul_full = np.cumsum(full_ev) # kumulativní součet
full_ev[:15]

In [ ]:
percent = 2
part = int(percent/100. * xyz.shape[0])

# TODO: part_cor, part_ev ... totéž pro xyz_flat[:part]



In [ ]:

# ODOT

part_ev[:15]

In [ ]:
# končíme na 95%, dál už není nic zajímavého
show = np.sum(cumul_full < 0.95)
show

In [ ]:
plt.figure(figsize=(14,5))
plt.plot(cumul_part[:show],label='part',marker='x')
plt.plot(cumul_full[:show],label='full',marker='o')
plt.legend()
plt.show()

**A to je konec pohádky. Dobrou noc, strýčku Fido, dobrou noc, děti.**